# Containers

#### Docker implements the Open Container Initiative (OCI) specifications. This means everything you learn in this chapter applies to other container runtimes and platforms that implement the OCI specifications.

#### A container is the runtime instance of an image. In the same way that you can start a virtual machine (VM) from a virtual machine template, you start one or more containers from a single image. The big difference between a VM and a container is that containers are smaller, faster, and more portable.

#### The simplest way to start a container is with the docker run command. The command can take a lot of arguments, but in its most basic form you tell it an image to use and an app to run: docker run <image> <app>. The following command will start a new container based on the Ubuntu Linux image and start a Bash shell.

- $ docker run -it ubuntu /bin/bash` 
- The -it flags connect your current terminal window to the container’s shell.
- You can manually stop a running container with docker stop and restart it with docker start. To get rid of a container forever, you have to explicitly delete it with docker rm.


## Containers vs VMs

#### Containers and VMs both need a host to run on. The host can be anything from your laptop, a bare metal server in your data center, or an instance in the public cloud.
#### Let’s assume a requirement where your business has a single physical server that needs to run 4 business applications.
#### In the VM model, the physical server is powered on and the hypervisor boots. Once booted, the hypervisor claims all physical resources such as CPU, RAM, storage, and network cards. It then carves these hardware resources into virtual constructs that look smell and feel exactly like the real thing. It then packages them into a software construct called a virtual machine (VM). We take those VMs and install an operating system and application on each one.

## Things are a bit different in the container model.

#### The server is powered on and the OS boots. In this container model the host’s OS claims all hardware resources. Next you install a container engine such as Docker. The container engine then carves-up the OS resources (process tree, filesystem, network stack etc) and packages them into virtual operating systems called containers. Each container looks smells and feels just like a real OS. Inside of each container we run an application.

#### If we assume the same scenario of a single physical server needing to run 4 business applications, we’d carve the OS into 4 containers and run a single application inside each. 

#### At a high level, hypervisors perform hardware virtualization — they carve up physical hardware resources into virtual versions called VMs. On the other hand, containers perform OS virtualization — they carve OS resources into virtual versions called containers.

#### Another thing to consider is application start times. Containers start a lot faster than VMs because they only have to start the application — the kernel is already up and running on the host. In the VM model, each VM needs to boot a full OS before it can start the app.

#### This all makes the container model leaner and more efficient than the VM model. You can pack more applications onto less resources, start them faster, and pay less in licensing and admin costs, as well as present less of an attack surface!



## This command starts a simple container that will run a containerized version of Ubuntu Linux.
- docker run -it ubuntu:latest /bin/bash

#### docker run tells Docker to run a new container. The -it flags make the container interactive and attach it to your terminal. The ubuntu:latest argument tells Docker which image to use to start the container. Finally, /bin/bash is the application to run in the container.

#### When you hit Return, the Docker client packages up the command and POSTs it to the API server running on the Docker daemon. The Docker daemon accepts the command and searches the host’s local image repository to see if it already has a copy of the image. In our example it didn’t, so it went to Docker Hub to see if it could find it there. It found it, pulled it locally, and stored it in the local cache.

#### Typing exit while logged in to the container will terminate the Bash process and the whole container will exit (terminate). This is because a container cannot exist without its designated main process. This is true of Linux and Windows containers — killing the main process in the container will kill the container.

- $ docker run --name percy -it ubuntu:latest /bin/bash
root@9cb2d2fd1d65:/#

#### That’s the container created, and we named it “percy” for persistent. Now let’s put it to work by writing some data to it. The following procedure writes some text to a new file in the /tmp directory and verifies the operation succeeded. Be sure to run these commands from within the container you just started.

- root@9cb2d2fd1d65:/# cd tmp

- root@9cb2d2fd1d65:/tmp# ls -l
total 0

- root@9cb2d2fd1d65:/tmp# echo "Sunderland is the greatest football team in the world" > newfile

- root@9cb2d2fd1d65:/tmp# ls -l
total 4

- -rw-r--r-- 1 root root 14 Apr 27 11:22 newfile

- root@9cb2d2fd1d65:/tmp# cat newfile
Sunderland is the greatest football team in the world

#### The data created in this example is stored on the Docker hosts local filesystem. If the Docker host fails, the data will be lost.
#### Containers are designed to be immutable objects and it’s not a good practice to write data to them.

#### For these reasons, Docker provides volumes. These exist outside of containers but can be mounted into them.

## Stopping containers gracefully

#### In the previous example, the container was running the /bin/bash app. When you kill a running container with docker rm <container> -f, the container is killed immediately without warning. You’re literally giving the container, and the app it’s running, no chance to complete any operations and gracefully exit.

#### However, the docker stop command is far more polite. It gives the process inside of the container ~10 seconds to complete any final tasks and gracefully shutdown. Once the command completes, you can delete the container with docker rm.

#### Behind the scenes the docker stop command sends a SIGTERM signal to the main application process inside the container (PID 1). This is a request to terminate and gives the process a chance to clean things up and gracefully shut itself down. If it’s still running after 10 seconds it will be issued a SIGKILL which terminates it with force.

#### Restart policies are applied per-container. They can be configured imperatively on the command line as part of docker run commands, or declaratively in YAML files for use with higher-level tools such as Docker Swarm, Docker Compose, and Kubernetes. At the time of writing, the following restart policies exist:

- always
- unless-stopped
- on-failure


#### The always policy is the simplest. It always restarts a failed container unless it’s been explicitly stopped. An easy way to demonstrate this is to start a new interactive container with the --restart always policy and tell it to run a shell process. When the container starts you’ll be automatically attached to its shell. Typing exit from the shell will kill the container’s PID 1 process and kill the container. However, Docker will automatically restart it because it has the --restart always policy. If you issue a docker ps command, you’ll see that the container’s uptime is less than the time since it was created. Let’s put it to the test.

- docker run --name neversaydie -it --restart always alpine sh

#### An interesting feature of the --restart always policy is that if you stop a container with docker stop and then restart the Docker daemon, the container will be restarted. To be clear… you start a new container with the --restart always policy and then intentionally stop it with the docker stop command. At this point the container is in the Stopped (Exited) state. However, if you restart the Docker daemon, the container will be automatically restarted when the daemon comes back up. You need to be aware of this.

#### The main difference between the always and unless-stopped policies is that containers with the --restart unless-stopped policy will not be restarted when the daemon restarts if they were in the Stopped (Exited) state. That might be a confusing sentence, so let’s walk through an example.

#### The on-failure policy will restart a container if it exits with a non-zero exit code. It will also restart containers when the Docker daemon restarts, even ones that were in the stopped state.


#### If you are working with Docker Compose or Docker Stacks, you can apply the restart policy to a service object as follows. We’ll talk more about these technologies later in the book.


services:
  myservice:
    <Snip>
    restart_policy:
      condition: always | unless-stopped | on-failure


- docker run -d --name webserver -p 80:8080 \
  nigelpoulton/ddd-book:web0.1

#### Notice that your shell prompt hasn’t changed. This is because the container was started in the background with the -d flag. Starting a container like this doesn’t attach it to your terminal.

#### We know docker run starts a new container. However, this time we give it the -d flag instead of -it. -d stands for detached or daemon mode and tells the container to run in the background. You can’t use the -d and -it flags in the same command.

#### After that, the command names the container “webserver”. The -p flag maps port 80 on the Docker host to port 8080 inside the container. This means that traffic hitting the Docker host on port 80 will be directed to port 8080 inside of the container. The image we’re using for this container contains a web service that listens on port 8080. This means the container will come up running a web server listening on port 8080.

#### Once the container is running, a docker ps command will show the container as running and the ports that are mapped. It’s important to know that port mappings are expressed as host-port:container-port.